In [1]:
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()

In [2]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

In [3]:
def normalize(text):
    tokens = tokenizer.tokenize(text.lower())
    return tokens

In [4]:
import itertools

In [5]:
def get_ngrams(tokens, n=2, patterns=None, stoplist=[]):
    tokens = list(filter(lambda x: x not in stoplist, tokens))
    lemmas = [m.parse(t)[0].normal_form for t in tokens]
    ngrams = []
    for i in range(len(lemmas) - (n-1)):
        ngram = lemmas[i:i+n]
        tngram = tokens[i:i+n]
        tags = [m.parse(t)[0].tag.POS for t in tngram]
        if patterns is not None:
            if patterns == ['VERB', 'NOUN'] and tags in patterns:
                c = 0
                for t in lemmas:
                    if t == ngram[0]:
                        c += 1
                if c > 49 and ('accs' in m.parse(tokens[i+n])[0].tag or 'accs' in m.parse(tokens[i+n])[2].tag or 'accs' in m.parse(tokens[i+n])[1].tag):
                    ngrams.append(tuple(ngram))
            elif tags in patterns:
                ngrams.append(tuple(ngram))
        else:
            ngrams.append(tuple(ngram))
    return ngrams

In [6]:
!/Users/masha/Desktop/need_for_vyshka/P_for_Proga/python/udpipe-1.2.0-bin/bin-osx/udpipe --input horizontal --output conllu \
--tokenize --tag --parse \
/Users/masha/Desktop/need_for_vyshka/P_for_Proga/python/udpipe-1.2.0-bin/models/russian-syntagrus-ud-2.4-190531.udpipe \
< testset2.txt > testset2.conllu

Loading UDPipe model: done.


In [7]:
from nltk.parse import DependencyGraph

In [8]:
trees = []

with open('testset2.conllu') as f:
    parsed_sents = f.read().split('\n\n')

    for sent in parsed_sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))

In [9]:
print(trees[1163])

1	В	в	ADP	_	_	2	case	_	_
2	ответ	ответ	NOUN	_	Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing	5	obl	_	_
3	менеджмент	менеджмент	NOUN	_	Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing	5	nsubj	_	_
4	Ford	Форд	PROPN	_	Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing	3	appos	_	_
5	предложил	предложить	VERB	_	Aspect=Perf|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	_
6	им	они	PRON	_	Case=Dat|Number=Plur|Person=3	5	iobj	_	_
7	повышение	повышение	NOUN	_	Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing	5	obj	_	_
8	зарплат	зарплата	NOUN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Plur	7	nmod	_	_
9	на	на	ADP	_	_	13	case	_	_
10	14	14	NUM	_	_	13	nummod	_	SpaceAfter=No
11	–	–	PUNCT	_	_	13	punct	_	SpaceAfter=No
12	20	20	NUM	_	_	13	nummod	_	SpaceAfter=No
13	%	%	SYM	_	_	7	nmod	_	SpaceAfter=No
14	,	,	PUNCT	_	_	17	punct	_	_
15	один	один	NUM	_	Case=Nom|Gender=Masc	17	nummod:gov	_	_
16	дополнительный	дополнительный	ADJ	_	Animacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|Number=Sing	17	amod	_

In [10]:
col = []
for i, tree in enumerate(trees):
    try:
        for tri in DependencyGraph(trees[i], top_relation_label='root').triples():
            if tri[0][-1] == 'VERB' and 'obj' in tri[1] and tri[2][-1] == 'NOUN':
                b = [m.parse(t)[0].normal_form for t in [tri[0][0], tri[2][0]]]
                if b not in col:
                    col.append(b)
    except AssertionError:
        continue
    except TypeError:
        continue

In [11]:
col

[['подать', 'иск'],
 ['признать', 'договор'],
 ['запускать', 'механизм'],
 ['сообщить', 'корреспондент'],
 ['рассмотреть', 'вопрос'],
 ['терпеть', 'бедствие'],
 ['определить', 'срок'],
 ['оберегать', 'покоить'],
 ['обвинить', 'руководство'],
 ['вызвать', 'недовольство'],
 ['привести', 'обитатель'],
 ['гласить', 'постановление'],
 ['приносить', 'польза'],
 ['получить', 'срок'],
 ['собирать', 'доказательство'],
 ['обвинить', 'олигарх'],
 ['предать', 'суд'],
 ['предать', 'рота'],
 ['передаться', 'судья'],
 ['оплачивать', 'обязанность'],
 ['выиграть', 'суд'],
 ['направить', 'ведомство'],
 ['девоенизировать', 'суд'],
 ['просить', 'суд'],
 ['сделать', 'заявление'],
 ['рассматривать', 'ходатайство'],
 ['затруднять', 'доступ'],
 ['отказать', 'жалоба'],
 ['отозвать', 'иск'],
 ['сделать', 'шаг'],
 ['предоставлять', 'норвежец'],
 ['предоставлять', 'дать'],
 ['затевать', 'преследование'],
 ['вручить', 'фигурант'],
 ['утвердить', 'заключение'],
 ['иметь', 'вид'],
 ['избрать', 'мера'],
 ['оказать', 

In [12]:
with open('testset2.txt', 'r', encoding='utf8') as file:  
    text = file.read()  

In [13]:
from collections import Counter

In [14]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /Users/masha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
tokens = normalize(text)

In [16]:
bigrams = get_ngrams(tokens, patterns=[['VERB', 'NOUN']], stoplist=stop)
c = Counter()
for n in bigrams:
    c[n] += 1

In [17]:
c.most_common(10)

[(('подать', 'иск'), 51),
 (('удовлетворить', 'иск'), 44),
 (('обратиться', 'суд'), 38),
 (('подать', 'суд'), 35),
 (('принять', 'решение'), 35),
 (('вынести', 'решение'), 34),
 (('попросить', 'суд'), 25),
 (('удовлетворить', 'ходатайство'), 24),
 (('оставить', 'сила'), 22),
 (('иметь', 'право'), 21)]

metrics

In [18]:
import nltk
from nltk.collocations import *

In [19]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder2 = BigramCollocationFinder.from_documents([[m.parse(t)[0].normal_form for t in tokens]])

In [20]:
finder2.nbest(bigram_measures.likelihood_ratio, 20)

[('арбитражный', 'суд'),
 ('суд', 'москва'),
 ('верховный', 'суд'),
 ('уголовный', 'дело'),
 ('миллион', 'рубль'),
 ('2006', 'год'),
 ('лишение', 'свобода'),
 ('под', 'стража'),
 ('миллиард', 'рубль'),
 ('один', 'из'),
 ('платон', 'лебедев'),
 ('тот', 'что'),
 ('2007', 'год'),
 ('тысяча', 'рубль'),
 ('2005', 'год'),
 ('а', 'также'),
 ('михаил', 'ходорковский'),
 ('2004', 'год'),
 ('басманный', 'суд'),
 ('борис', 'березовский')]

In [21]:
finder2.nbest(bigram_measures.pmi, 20)

[('00', 'мск'),
 ('112', '835'),
 ('127', 'томный'),
 ('191', '350'),
 ('2013', '2014'),
 ('228', 'денежка'),
 ('2339', '321'),
 ('2513', '427'),
 ('3087', '455'),
 ('314', '194'),
 ('366', '754'),
 ('390', 'гектар'),
 ('396', 'метровый'),
 ('571', '406'),
 ('697', '577'),
 ('787', 'dreamliner'),
 ('81', '205'),
 ('900ex', 'easy'),
 ('968', '991'),
 ('99', '94')]

In [22]:
dice = finder2.score_ngrams(bigram_measures.dice)

In [23]:
sorted([x for x in dice if x[1] != 1.0], key=lambda x: x[1], reverse=True)[:20]

[(('матросский', 'тишина'), 0.9473684210526315),
 (('нью', 'йорк'), 0.9333333333333333),
 (('p', 'i'), 0.9230769230769231),
 (('северо', 'кавказский'), 0.9230769230769231),
 (('антонио', 'вальдес'), 0.9090909090909091),
 (('вальдес', 'гарсиа'), 0.9090909090909091),
 (('пугинский', 'афанасьев'), 0.9090909090909091),
 (('сцс', 'совинтел'), 0.9090909090909091),
 (('joy', 'lud'), 0.9),
 (('истый', 'бридж'), 0.8888888888888888),
 (('тимур', 'сапир'), 0.8888888888888888),
 (('бадри', 'патаркацишвили'), 0.88),
 (('нарый', 'фоминский'), 0.875),
 (('нессим', 'гаон'), 0.875),
 (('alltunes', 'com'), 0.8571428571428571),
 (('b', 'v'), 0.8571428571428571),
 (('bank', 'of'), 0.8571428571428571),
 (('nuclear', 'services'), 0.8571428571428571),
 (('resourses', 'b'), 0.8571428571428571),
 (('services', 'and'), 0.8571428571428571)]

In [24]:
nlog = []
ndice = []
npmi = []

In [25]:
t = 2750
while len(nlog) < 100:
    for b in finder2.nbest(bigram_measures.likelihood_ratio, t):
        if b not in nlog:
            if [b[0], b[-1]] in col:
                nlog.append(b)
    t += 20

In [26]:
len(nlog)

100

In [27]:
t = 2750
while len(npmi) < 100:
    for b in finder2.nbest(bigram_measures.pmi, t):
        if b not in npmi:
            if [b[0], b[-1]] in col:
                npmi.append(b)
    t += 20

In [28]:
len(npmi)

100

In [29]:
t = 0
while len(ndice) < 100:
    if t == 0:
        t2 = 2750
    else:
        t2 = t+20
    for b in sorted([x for x in dice if x[1] != 1.0], key=lambda x: x[1], reverse=True)[t:t2]:
        if b[0] not in ndice:
            if [b[0][0], b[0][-1]] in col:
                ndice.append(b[0])
    if t == 0:
        t = 2750
    else:
        t += 20

In [30]:
len(ndice)

100

Gold

In [31]:
nlog[:10]

[('подать', 'иск'),
 ('удовлетворить', 'иск'),
 ('представлять', 'интерес'),
 ('принять', 'решение'),
 ('выдать', 'санкция'),
 ('вынести', 'решение'),
 ('удовлетворить', 'ходатайство'),
 ('обжаловать', 'решение'),
 ('иметь', 'право'),
 ('наложить', 'арест')]

In [32]:
ndice[0:10]

[('попирать', 'презумпция'),
 ('внедрить', 'религиоведение'),
 ('возродить', 'выборность'),
 ('выесть', 'яйцо'),
 ('дискредитировать', 'добросовестность'),
 ('обнаружиться', 'потребность'),
 ('обслуживающий', 'геев'),
 ('опечатать', 'торгцентр'),
 ('опустошить', 'карман'),
 ('пожелать', 'новосёл')]

In [33]:
npmi[0:10]

[('воплощать', 'бытие'),
 ('заделывать', 'ямища'),
 ('запускать', 'механизм'),
 ('заставлять', 'дорожник'),
 ('нитрохема', 'дистрибьюшн'),
 ('облагородить', 'капитализм'),
 ('ослепить', 'фара'),
 ('оставлять', 'вакуум'),
 ('открыть', 'шлюз'),
 ('охладить', 'пыл')]

In [34]:
gold = set(ndice) & set(npmi) & set(nlog)

In [35]:
gold

{('направлять', 'излишек'),
 ('обосновывать', 'предположение'),
 ('обслуживающий', 'геев'),
 ('повысить', 'популярность'),
 ('попирать', 'презумпция'),
 ('проявить', 'гуманизм'),
 ('регламентировать', 'переезд')}

In [36]:
with open('verb_coll.txt', 'r', encoding='utf8') as file:  
    diction = file.readlines()

In [37]:
d = []
for i in diction:
    ts = normalize(i.replace('\n', '').split('\t')[-1])
    ls = [m.parse(t)[0].normal_form for t in ts]
    if len(ls) < 3:
        d.append(ls)

In [38]:
d

[['аборт', 'делать'],
 ['делать', 'аборт'],
 ['сделать', 'аборт'],
 ['авария', 'произойти'],
 ['авария', 'случиться'],
 ['потерпеть', 'авария'],
 ['произойти', 'авария'],
 ['случиться', 'авария'],
 ['завоевать', 'авторитет'],
 ['подорвать', 'авторитет'],
 ['подорвать', 'авторитет'],
 ['пользоваться', 'авторитет'],
 ['оспаривать', 'акт'],
 ['активность', 'проявлять'],
 ['активность', 'проявлять'],
 ['проявить', 'активность'],
 ['проявить', 'активность'],
 ['проявлять', 'активность'],
 ['проявлять', 'активность'],
 ['акцент', 'делаться'],
 ['акцент', 'переноситься'],
 ['акцент', 'сделать'],
 ['делать', 'акцент'],
 ['расставлять', 'акцент'],
 ['расставлять', 'акцент'],
 ['расставлять', 'акцент'],
 ['сделать', 'акцент'],
 ['сместить', 'акцент'],
 ['сместить', 'акцент'],
 ['предлагать', 'алгоритм'],
 ['предлагать', 'алгоритм'],
 ['разработать', 'алгоритм'],
 ['разработать', 'алгоритм'],
 ['вызывать', 'аллергия'],
 ['вызывать', 'аллергия'],
 ['найти', 'альтернатива'],
 ['найти', 'альтернатив

In [43]:
gold = []
for g in set(ndice) & set(npmi) & set(nlog):
    print([g[0], g[-1]])
    if [g[0], g[-1]] in d:
        gold.append(g)

['обосновывать', 'предположение']
['проявить', 'гуманизм']
['повысить', 'популярность']
['направлять', 'излишек']
['попирать', 'презумпция']
['обслуживающий', 'геев']
['регламентировать', 'переезд']


In [44]:
gold

[]